In [1]:
import pandas as pd
import numpy as np

from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os
from nltk import tokenize 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#2. Get the file
data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'

In [0]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append(f'{codes_path}')

In [5]:
import models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

train['mal']    = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [0]:
max_features = 40000
maxlen       = 400
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


max_sen_len    = 100
max_sent_amount = 4

seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

# LSTM

In [8]:
models.train_model(X=X, y=y, mtype = 'LSTM', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0


W0805 09:26:59.159408 140230199809920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 09:26:59.161224 140230199809920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 09:26:59.173749 140230199809920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0805 09:26:59.193707 140230199809920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0805 09:26:59.202605 

Fitting
31745/31745 [==============================] - 6s 183us/step
 0.3 fold 0 precision 0.852 recall 0.754 fscore 0.8
fold 0 average precision 0.886
fold 0 roc auc 0.976
 fold 1
Fitting
31745/31745 [==============================] - 2s 78us/step
 0.3 fold 1 precision 0.883 recall 0.731 fscore 0.8
fold 1 average precision 0.884
fold 1 roc auc 0.973
 fold 2
Fitting
31743/31743 [==============================] - 2s 77us/step
 0.3 fold 2 precision 0.846 recall 0.784 fscore 0.814
fold 2 average precision 0.888
fold 2 roc auc 0.973
 fold 3
Fitting
31743/31743 [==============================] - 2s 77us/step
 0.3 fold 3 precision 0.625 recall 0.845 fscore 0.719
fold 3 average precision 0.835
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 2s 78us/step
 0.3 fold 4 precision 0.562 recall 0.89 fscore 0.689
fold 4 average precision 0.806
fold 4 roc auc 0.97
PR-C 0.86
ROC AUC 0.972
FScore 0.764


In [9]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'LSTM', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 5s 76us/step
_________________________________
PR-C is 0.855
_________________________________

_________________________________
ROC AUC is 0.972
_________________________________
_________________________________
FScore is 0.767
_________________________________



# GRU

In [10]:
models.train_model(X=X, y=y, mtype = 'GRU', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 2s 73us/step
 0.3 fold 0 precision 0.838 recall 0.772 fscore 0.804
fold 0 average precision 0.887
fold 0 roc auc 0.975
 fold 1
Fitting
31745/31745 [==============================] - 2s 72us/step
 0.3 fold 1 precision 0.894 recall 0.714 fscore 0.794
fold 1 average precision 0.887
fold 1 roc auc 0.973
 fold 2
Fitting
31743/31743 [==============================] - 2s 72us/step
 0.3 fold 2 precision 0.884 recall 0.75 fscore 0.812
fold 2 average precision 0.894
fold 2 roc auc 0.977
 fold 3
Fitting
31743/31743 [==============================] - 2s 72us/step
 0.3 fold 3 precision 0.627 recall 0.855 fscore 0.724
fold 3 average precision 0.834
fold 3 roc auc 0.971
 fold 4
Fitting
31743/31743 [==============================] - 2s 72us/step
 0.3 fold 4 precision 0.52 recall 0.921 fscore 0.665
fold 4 average precision 0.799
fold 4 roc auc 0.97
PR-C 0.86
ROC AUC 0.973
FScore 0.76


In [11]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'GRU', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 5s 70us/step
_________________________________
PR-C is 0.859
_________________________________

_________________________________
ROC AUC is 0.973
_________________________________
_________________________________
FScore is 0.768
_________________________________



# CNN

In [12]:
models.train_model(X=X, y=y, mtype = 'CNN', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 1s 44us/step
 0.3 fold 0 precision 0.892 recall 0.698 fscore 0.783
fold 0 average precision 0.882
fold 0 roc auc 0.975
 fold 1
Fitting
31745/31745 [==============================] - 1s 39us/step
 0.3 fold 1 precision 0.834 recall 0.765 fscore 0.798
fold 1 average precision 0.881
fold 1 roc auc 0.974
 fold 2
Fitting
31743/31743 [==============================] - 1s 42us/step
 0.3 fold 2 precision 0.866 recall 0.741 fscore 0.799
fold 2 average precision 0.882
fold 2 roc auc 0.974
 fold 3
Fitting
31743/31743 [==============================] - 1s 40us/step
 0.3 fold 3 precision 0.625 recall 0.863 fscore 0.725
fold 3 average precision 0.829
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 1s 40us/step
 0.3 fold 4 precision 0.49 recall 0.926 fscore 0.641
fold 4 average precision 0.787
fold 4 roc auc 0.967
PR-C 0.852
ROC AUC 0.972
FScore 0.749


In [13]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'CNN', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 3s 39us/step
_________________________________
PR-C is 0.85
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________
_________________________________
FScore is 0.762
_________________________________



# BGRU

In [14]:
models.train_model(X=X, y=y, mtype = 'BGRU', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 4s 139us/step
 0.3 fold 0 precision 0.817 recall 0.793 fscore 0.805
fold 0 average precision 0.89
fold 0 roc auc 0.977
 fold 1
Fitting
31745/31745 [==============================] - 4s 138us/step
 0.3 fold 1 precision 0.821 recall 0.789 fscore 0.805
fold 1 average precision 0.887
fold 1 roc auc 0.975
 fold 2
Fitting
31743/31743 [==============================] - 4s 137us/step
 0.3 fold 2 precision 0.875 recall 0.759 fscore 0.813
fold 2 average precision 0.892
fold 2 roc auc 0.974
 fold 3
Fitting
31743/31743 [==============================] - 4s 137us/step
 0.3 fold 3 precision 0.66 recall 0.834 fscore 0.737
fold 3 average precision 0.833
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 4s 137us/step
 0.3 fold 4 precision 0.574 recall 0.881 fscore 0.695
fold 4 average precision 0.805
fold 4 roc auc 0.971
PR-C 0.861
ROC AUC 0.973
FScore 0.771


In [15]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 9s 135us/step
_________________________________
PR-C is 0.86
_________________________________

_________________________________
ROC AUC is 0.973
_________________________________
_________________________________
FScore is 0.758
_________________________________



# BLSTM

In [16]:
models.train_model(X=X, y=y, mtype = 'BLSTM', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 5s 149us/step
 0.3 fold 0 precision 0.877 recall 0.73 fscore 0.797
fold 0 average precision 0.882
fold 0 roc auc 0.971
 fold 1
Fitting
31745/31745 [==============================] - 5s 149us/step
 0.3 fold 1 precision 0.922 recall 0.617 fscore 0.739
fold 1 average precision 0.845
fold 1 roc auc 0.956
 fold 2
Fitting
31743/31743 [==============================] - 5s 149us/step
 0.3 fold 2 precision 0.796 recall 0.811 fscore 0.804
fold 2 average precision 0.889
fold 2 roc auc 0.974
 fold 3
Fitting
31743/31743 [==============================] - 5s 150us/step
 0.3 fold 3 precision 0.67 recall 0.827 fscore 0.74
fold 3 average precision 0.836
fold 3 roc auc 0.969
 fold 4
Fitting
31743/31743 [==============================] - 5s 149us/step
 0.3 fold 4 precision 0.524 recall 0.912 fscore 0.665
fold 4 average precision 0.795
fold 4 roc auc 0.969
PR-C 0.85
ROC AUC 0.968
FScore 0.749


In [17]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BLSTM', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 10s 147us/step
_________________________________
PR-C is 0.852
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________
_________________________________
FScore is 0.748
_________________________________



# BGRU + Max Pooling

In [18]:
models.train_model(X=X, y=y, mtype = 'BGRU_max', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 5s 158us/step
 0.3 fold 0 precision 0.859 recall 0.739 fscore 0.794
fold 0 average precision 0.887
fold 0 roc auc 0.976
 fold 1
Fitting
31745/31745 [==============================] - 5s 156us/step
 0.3 fold 1 precision 0.846 recall 0.757 fscore 0.799
fold 1 average precision 0.883
fold 1 roc auc 0.973
 fold 2
Fitting
31743/31743 [==============================] - 5s 157us/step
 0.3 fold 2 precision 0.796 recall 0.808 fscore 0.802
fold 2 average precision 0.889
fold 2 roc auc 0.975
 fold 3
Fitting
31743/31743 [==============================] - 5s 157us/step
 0.3 fold 3 precision 0.611 recall 0.86 fscore 0.714
fold 3 average precision 0.834
fold 3 roc auc 0.969
 fold 4
Fitting
31743/31743 [==============================] - 5s 157us/step
 0.3 fold 4 precision 0.513 recall 0.919 fscore 0.658
fold 4 average precision 0.803
fold 4 roc auc 0.97
PR-C 0.859
ROC AUC 0.973
FScore 0.754


In [19]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU_max', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 10s 154us/step
_________________________________
PR-C is 0.857
_________________________________

_________________________________
ROC AUC is 0.972
_________________________________
_________________________________
FScore is 0.764
_________________________________



# BGRU + attention

In [20]:
models.train_model(X=X, y=y, mtype = 'BGRU_att', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 6s 197us/step
 0.3 fold 0 precision 0.852 recall 0.763 fscore 0.805
fold 0 average precision 0.887
fold 0 roc auc 0.972
 fold 1
Fitting
31745/31745 [==============================] - 6s 197us/step
 0.3 fold 1 precision 0.9 recall 0.699 fscore 0.787
fold 1 average precision 0.878
fold 1 roc auc 0.969
 fold 2
Fitting
31743/31743 [==============================] - 6s 195us/step
 0.3 fold 2 precision 0.866 recall 0.725 fscore 0.789
fold 2 average precision 0.877
fold 2 roc auc 0.971
 fold 3
Fitting
31743/31743 [==============================] - 6s 196us/step
 0.3 fold 3 precision 0.617 recall 0.858 fscore 0.718
fold 3 average precision 0.822
fold 3 roc auc 0.968
 fold 4
Fitting
31743/31743 [==============================] - 6s 196us/step
 0.3 fold 4 precision 0.567 recall 0.88 fscore 0.689
fold 4 average precision 0.792
fold 4 roc auc 0.968
PR-C 0.851
ROC AUC 0.97
FScore 0.758


In [21]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU_att', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 13s 194us/step
_________________________________
PR-C is 0.856
_________________________________

_________________________________
ROC AUC is 0.972
_________________________________
_________________________________
FScore is 0.757
_________________________________



# BGRU + Average Pooling

In [22]:
models.train_model(X=X, y=y, mtype = 'BGRU_avg', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 5s 156us/step
 0.3 fold 0 precision 0.85 recall 0.771 fscore 0.808
fold 0 average precision 0.89
fold 0 roc auc 0.976
 fold 1
Fitting
31745/31745 [==============================] - 5s 156us/step
 0.3 fold 1 precision 0.84 recall 0.758 fscore 0.797
fold 1 average precision 0.879
fold 1 roc auc 0.971
 fold 2
Fitting
31743/31743 [==============================] - 5s 157us/step
 0.3 fold 2 precision 0.87 recall 0.771 fscore 0.817
fold 2 average precision 0.895
fold 2 roc auc 0.976
 fold 3
Fitting
31743/31743 [==============================] - 5s 157us/step
 0.3 fold 3 precision 0.675 recall 0.822 fscore 0.742
fold 3 average precision 0.836
fold 3 roc auc 0.97
 fold 4
Fitting
31743/31743 [==============================] - 5s 155us/step
 0.3 fold 4 precision 0.591 recall 0.864 fscore 0.702
fold 4 average precision 0.793
fold 4 roc auc 0.966
PR-C 0.859
ROC AUC 0.972
FScore 0.773


In [23]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'BGRU_avg', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 10s 155us/step
_________________________________
PR-C is 0.855
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________
_________________________________
FScore is 0.769
_________________________________



# psHAN

In [24]:
models.train_model(X=X, y=y, mtype = 'psHAN', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
Fitting
31745/31745 [==============================] - 5s 165us/step
 0.3 fold 0 precision 0.871 recall 0.72 fscore 0.788
fold 0 average precision 0.879
fold 0 roc auc 0.971
 fold 1
Fitting
31745/31745 [==============================] - 5s 165us/step
 0.3 fold 1 precision 0.864 recall 0.736 fscore 0.795
fold 1 average precision 0.88
fold 1 roc auc 0.971
 fold 2
Fitting
31743/31743 [==============================] - 5s 165us/step
 0.3 fold 2 precision 0.795 recall 0.813 fscore 0.804
fold 2 average precision 0.886
fold 2 roc auc 0.972
 fold 3
Fitting
31743/31743 [==============================] - 5s 165us/step
 0.3 fold 3 precision 0.735 recall 0.769 fscore 0.752
fold 3 average precision 0.829
fold 3 roc auc 0.969
 fold 4
Fitting
31743/31743 [==============================] - 5s 164us/step
 0.3 fold 4 precision 0.625 recall 0.816 fscore 0.708
fold 4 average precision 0.791
fold 4 roc auc 0.968
PR-C 0.853
ROC AUC 0.97
FScore 0.769


In [25]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'psHAN', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

Fitting
64830/64830 [==============================] - 11s 162us/step
_________________________________
PR-C is 0.853
_________________________________

_________________________________
ROC AUC is 0.971
_________________________________
_________________________________
FScore is 0.768
_________________________________



# HAN

In [0]:
#3. Read file as panda dataframe
train           = pd.read_csv(f'{data_path}/train_cleaned_sent_punkt.csv') 
test_labelled   = pd.read_csv(f'{data_path}/test_labelled_cleaned_sent_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_sent_punkt.csv') 

train['mal']    = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [27]:
models.train_model(X=X, y=y, mtype = 'HAN', cv=True, nfolds=5, epochs=4, cv_models_path=cv_models_path, train=False, rs=rs)

 fold 0
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
Fitting
31745/31745 [==============================] - 5s 166us/step
 0.3 fold 0 precision 0.815 recall 0.768 fscore 0.791
fold 0 average precision 0.87
fold 0 roc auc 0.965
 fold 1
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
Fitting
31745/31745 [==============================] - 5s 165us/step
 0.3 fold 1 precision 0.763 recall 0.791 fscore 0.777
fold 1 average precision 0.863
fold 1 roc auc 0.966
 fold 2
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
Fitting
31743/31743 [==============================] - 5s 165us/step
 0.3 fold 2 precision 0.88 recall 0.703 fscore 0.782
fold 2 average precision 0.866
fold 2 roc auc 0.964
 fold 3
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
Fitting
31743/31743 [==============================] - 5s

In [28]:
models.train_model(X=X, y=y, X_test=X_test, y_test=y_test, mtype = 'HAN', cv=False, nfolds=5, epochs=4, cv_models_path=models_path, train=False, rs=rs)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
0
10000
20000
30000
40000
50000
60000
Fitting
64830/64830 [==============================] - 11s 163us/step
_________________________________
PR-C is 0.833
_________________________________

_________________________________
ROC AUC is 0.964
_________________________________
_________________________________
FScore is 0.748
_________________________________

